In [ ]:
# Some jupyter notebook magic to reload modules automatically when they change
# not necessary for this specific notebook but useful in general
%load_ext autoreload
%autoreload 2

# Gives you high resolution images within the notebook
%config InlineBackend.figure_format = 'retina'

In [ ]:
from definitions import ROOT_DIR
import os
from pypsdm import GridContainer

# The PSDM specific input models can be imported from the pypsdm.models.input and
# pypsdm.models.result. The `GridWithResults` container is located in pypsdm.models.gwr
from pypsdm.models.gwr import GridWithResults

grid_path = os.path.join(ROOT_DIR, "tests", "resources", "simple_grid", "input")
# IO data models in general have a from_csv method to parse psdm files
gwr = GridContainer.from_csv(grid_path)

# Output directory
target_grid_path = os.path.join(ROOT_DIR, "output", "enriched_grid")
if not os.path.exists(target_grid_path):
    os.makedirs(target_grid_path)

In [ ]:
from pypsdm.io.utils import delete_all_files_in_directory

delete_all_files_in_directory(target_grid_path)

In [ ]:
# input parameter, so far we only support to sample p_rated of the EVs in a normal distribution.
# EVs will have a storage size of 120.0 kWh and a consumption of 0.18 kWh/km each
# The EVCS will have a charging power of 22.0 kW and two charging points.
ev_p_rated_mean = 22.0
ev_p_rated_sigma = 0.0
ev_p_rated_min = 22.0
ev_p_rated_max = 22.0
ev_p_rated_params = [ev_p_rated_mean, ev_p_rated_sigma, ev_p_rated_min, ev_p_rated_max]
evcs_v2g = False

In [ ]:
# identify nodes to which should be connected. For example, customer nodes most probably have an existing load. So we're looking for nodes in lv grids with an existing load
lv_nodes = gwr.nodes.data[gwr.nodes.data["v_rated"] <= 1.0]
household_nodes = lv_nodes[lv_nodes.index.isin(gwr.loads.node)]
household_nodes.head()

In [ ]:
from pypsdm.models.input.create.enrich_grid import enrich_grid_by_evs_and_evcs
from pypsdm.models.input.create.participants import create_electric_vehicles
from pypsdm.models.input.create.participants import create_ev_charging_stations

# get dictionaries for the ev and evcs as well as DataFrames for POIs and POI Mapping
ev_dict, evcs_dict, df_poi, df_poi_map = enrich_grid_by_evs_and_evcs(
    nodes=household_nodes,
    ev_p_rated_params=ev_p_rated_params,
    evcs_v2g=evcs_v2g,
    controlling_em=False,
)

# with these the EVs and EVCSs can be created
evs = create_electric_vehicles(ev_dict)
evcs = create_ev_charging_stations(evcs_dict)

In [ ]:
from pypsdm.models.input.container.grid import GridContainer
from pypsdm.models.input.create.enrich_grid import add_pois_of_enriched_evcs

# we now can create an updated SystemParticipantsContainer and an updated node_participants_map
updated_participants = gwr.participants.copy(evs=evs, evcs=evcs)
node_participants = updated_participants.build_node_participants_map(gwr.raw_grid.nodes)
node_participants_map = updated_participants.build_node_participants_map(
    gwr.raw_grid.nodes
)

# with this we can build our updated grid containing the updated participants.
updated_grid = GridContainer(
    gwr.raw_grid, updated_participants, gwr.primary_data, node_participants_map
)

# Finally, we can write the updated grid as well as the updated POI and POI Mapping data to our output path
updated_grid.to_csv(target_grid_path, include_primary_data=True)
add_pois_of_enriched_evcs(df_poi, df_poi_map, target_grid_path)